# Tutorial for sampling using Qiskit
In this tutorial, we will look at tutorials of implementations using **Qiskit**. Participants are recommended to read the `sampling.ipynb` beforehand.

In QURI Parts, there are codes to convert **Qiskit** circuits and operators to **QURI Parts**. When implementing with **Qiskit**, you can use these codes to use the provided sampling function with qiskit circuits and operators. Let's actually implement it.

## ChallengeSampling

In [1]:
import sys

sys.path.append("../")
from utils.challenge_2024 import ChallengeSampling

# define challenge_sampling
challenge_sampling = ChallengeSampling()

   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environments/pyjuliapkg/Manifest.toml`


test mps sampling took: (24.355291843414307, Counter({2: 7, 0: 3}))


### Prepare a Qiskit circuit

In [2]:
from qiskit import QuantumCircuit as QiskitQuantumCircuit
from math import pi

qiskit_circuit = QiskitQuantumCircuit(4)
qiskit_circuit.x(0)
qiskit_circuit.h(1)
qiskit_circuit.y(2)
qiskit_circuit.cx(1, 2)
qiskit_circuit.rx(pi/4, 3)

You can use `circuit_from_qiskit()` to convert the qiskit circuit to the quri-parts circuit as follows:

In [3]:
from quri_parts.qiskit.circuit import circuit_from_qiskit

quri_parts_circuit = circuit_from_qiskit(qiskit_circuit)

# Sampler
 By using the instance challenge_sampling, you can construct the sampler.

In [4]:
# create sampler
sampler = challenge_sampling.create_sampler()
# possible to choose Qiksit circuits as an input
sampling_result = sampler(quri_parts_circuit, n_shots=1000)
print(f"counts: {sampling_result}")

   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environments/pyjuliapkg/Manifest.toml`


counts: Counter({3: 439, 5: 411, 13: 85, 11: 65})


A sampler receives two arguments: a circuit to be sampled and a number of repeated samplings (shots). The returned value is a mapping (dict) with the following keys and values:

- **Keys** Bitstrings of measurement outcomes, encoded as int. Each measured bit of each qubit are ordered from the least significant bit to the most significant bit. For example, when qubit 0 and qubit 2 are measured to be in  while the others are in , the bitstring is 0b0101.

- **Values** Counts of times for which each bitstring is measured. A sum of all counts is equal to the specified shots.

## Concurrent sampler
The concurrent sampler can take Qiksit circuits as input as same as the usual sampler.

In [5]:
# another circuit with 4 qubits
qiskit_circuit2 = QiskitQuantumCircuit(4)
qiskit_circuit2.t(0)
qiskit_circuit2.h(1)
qiskit_circuit2.ry(3*pi/4, 2)
qiskit_circuit2.cx(1, 2)
qiskit_circuit2.rz(pi, 3)

quri_parts_circuit2 = circuit_from_qiskit(qiskit_circuit2)

# create concurrent sampler
concurrent_sampler = challenge_sampling.create_concurrent_sampler()
concurrent_counts = concurrent_sampler([(quri_parts_circuit, 1000), (quri_parts_circuit2, 1000)])

print(f"concurrent_counts: {concurrent_counts}")

concurrent_counts: [Counter({3: 447, 5: 427, 13: 68, 11: 58}), Counter({2: 434, 4: 422, 0: 73, 6: 71})]


   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environments/pyjuliapkg/Manifest.toml`


# Sampling Estimator
In order to estimate an expectation value of operators involves operators and states, you need to use a sampling_estimator. You can construct the concurrent sampler as same as the usual sampler with some additional inputs. 

The sampling_estimator can take Qiksit circuits and Qiskit operators as inputs. First we define a qiskit operator to be estimated.

In [6]:
# define a qiskit operator to be estimated
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.opflow import PauliSumOp


qiskit_paulisumop = PauliSumOp(primitive=SparsePauliOp(Pauli("ZIII")), coeff=0.25)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IZZI")), coeff=2.0)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IXXI")), coeff=0.5 + 0.25j)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IZIY")), coeff=1.0j)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IIZY")), coeff=1.5 + 0.5j)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IXIY")), coeff=2.0j)
qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IIII")), coeff=3.0)
print(qiskit_paulisumop)

0.25 * ZIII
+ 2.0 * IZZI
+ (0.5+0.25j) * IXXI
+ 1j * IZIY
+ (1.5+0.5j) * IIZY
+ 2j * IXIY
+ 3.0 * IIII


/var/folders/vd/q1zmkbt90l9fsys7lk9sbbhm0000gn/T/ipykernel_16553/1631192179.py:6: DeprecationWarning: The class ``qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  qiskit_paulisumop = PauliSumOp(primitive=SparsePauliOp(Pauli("ZIII")), coeff=0.25)
/var/folders/vd/q1zmkbt90l9fsys7lk9sbbhm0000gn/T/ipykernel_16553/1631192179.py:7: DeprecationWarning: The class ``qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  qiskit_paulisumop += PauliSumOp(primitive=SparsePauliOp(Pauli("IZZI")), coeff=2.0)
/var/folders/vd/q1zmkbt90l9fsys7lk9sbbhm0000gn/T/ipykernel_16553/1631192179.py:8: DeprecationWarning: The class ``qiskit.opflow.primit

You can use `operator_from_qiskit_op()` to convert the qiskit operator to the quri-parts operator as follows:

In [7]:
from quri_parts.qiskit.operator import operator_from_qiskit_op

quri_parts_operator = operator_from_qiskit_op(qiskit_paulisumop)

### Estimate expectation value by sampling
With these inputs and qiskit operators and qiskit circuits, sampling estimation can be performed as follows:

In [8]:
from quri_parts.core.measurement import bitwise_commuting_pauli_measurement
from quri_parts.core.sampling.shots_allocator import create_equipartition_shots_allocator
from quri_parts.core.state import GeneralCircuitQuantumState

shots_allocator = create_equipartition_shots_allocator()
measurement_factory = bitwise_commuting_pauli_measurement

circuit_state = GeneralCircuitQuantumState(4, quri_parts_circuit)

# create estimator
estimator = challenge_sampling.create_sampling_estimator(
    n_shots=10000,
    measurement_factory=measurement_factory,
    shots_allocator=shots_allocator,
)

# estimate the expectation value
estimated_value = estimator(quri_parts_operator, circuit_state)

# returns real part of estimated value
print(f"estimated_value :{estimated_value.value.real} ")

estimated_value :1.2428 
